In [2]:
!pip install medmnist

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
 %cd /content/drive/MyDrive/NELDA

/content/drive/MyDrive/NELDA


#### Loading onnx model called melda

In [6]:
import pickle
import medmnist
from medmnist import INFO, Evaluator
from pathlib import Path
import torch.utils.data as data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from DeepGA.Operators import *
from DeepGA.EncodingClass import *
from DeepGA.Decoding import *
from DeepGA.DataReader import *
from DeepGA.DistributedTraining import *

In [21]:
import torch
import numpy as np

def predict_ensemble(data_input, feature_extractor, lda_model, gmm_model=None, threshold=0.9817):
    """
    Run inference using the exported joblib models.

    Args:
        data_input: Either a DataLoader, an (images, targets) batch, or a numpy/torch array of images
        feature_extractor: joblib Runtime DeepGA model for CNN feature extractor
        lda_model: joblib Runtime session for LDA classifier
        gmm_model: Optional GMM model for handling uncertain cases
        threshold: Confidence threshold for GMM processing

    Returns:
        labels: Original labels if available, else None
        predicted_labels: Model predictions including GMM novelty detection
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    feature_extractor = feature_extractor.to(device)
    feature_extractor.eval()  # Ensure model is in eval mode

    features = []
    labels = []

    with torch.no_grad():
        # Handle DataLoader-like input (has attribute dataset)
        if hasattr(data_input, 'dataset'):
            for images, targets in data_input:
                images = images.to(device)
                outputs = feature_extractor(images).squeeze()
                features.append(outputs.cpu().numpy())
                labels.append(targets.numpy())
            features = np.concatenate(features, axis=0)
            labels = np.concatenate(labels, axis=0)

        else:
            # Handle single batch input
            if isinstance(data_input, (tuple, list)) and len(data_input) == 2:
                images, targets = data_input
                if isinstance(images, np.ndarray):
                    images = torch.from_numpy(images).float()
                images = images.to(device)
                outputs = feature_extractor(images).squeeze()
                features = outputs.cpu().numpy()

                if isinstance(targets, torch.Tensor):
                    labels = targets.cpu().numpy()
                else:
                    labels = np.asarray(targets)
            else:
                # Single tensor/array of images
                if isinstance(data_input, np.ndarray):
                    images = torch.from_numpy(data_input).float()
                else:
                    images = data_input
                images = images.to(device)
                outputs = feature_extractor(images).squeeze()
                features = outputs.cpu().numpy()
                labels = None

    # Ensure features are 2D for sklearn
    if features.ndim > 2:
        features = features.reshape(features.shape[0], -1)

    # Run LDA model for prediction
    predicted_labels = lda_model.predict(features)
    probabilities = lda_model.predict_proba(features)
    reduced_features = lda_model.transform(features)

    # Process uncertain predictions with GMM if available
    if gmm_model is not None:
        idx_incorrectly_classified = []
        idx_correctly_classified = []

        # Identify uncertain predictions
        for i in range(len(probabilities)):
            if np.max(probabilities[i]) < threshold:
                idx_incorrectly_classified.append(i)
            else:
                idx_correctly_classified.append(i)

        # Process correctly classified samples
        labels_correctly_classified = [predicted_labels[i] for i in idx_correctly_classified]
        unique_labels = np.unique(labels_correctly_classified)
        n_labels_found = len(unique_labels)
        print("=======================   NELDA   =============================")
        print(f"Found {n_labels_found} unique labels in confident predictions")
        print(f"Labels founds: {unique_labels}")
        percentage_correct = (len(idx_correctly_classified)/len(predicted_labels))*100
        percentage_incorrect = (len(idx_incorrectly_classified)/len(predicted_labels))*100
        print(f"Confident labels found by NELDA represent the {percentage_correct} %")
        print(f"Untrusted tags found by NELDA represent the {percentage_incorrect} %")
        # Apply GMM if needed
        if n_labels_found < 12 and len(idx_incorrectly_classified) > 0:
            print(f"Domain adaptation: Analyzing data to identify clusters and assign labels.")
            missing_classes = 12 - n_labels_found
            print(f"NELDA info: Some of your data is not within the distribution of the data with which NELDA was trained. ")
            print(f" It was adjusted into {missing_classes} groups, and the following labels were assigned.")
            uncertain_features = reduced_features[idx_incorrectly_classified]
            gmm_model = GaussianMixture(n_components=missing_classes, init_params='kmeans', covariance_type='full')
            gmm_model.fit(uncertain_features)
            gmm_labels = gmm_model.predict(uncertain_features)
            new_labels = gmm_labels + 100  # Offset to avoid conflicts
            unique_new_labels = np.unique(new_labels)
            print(f"Labels assigned: {unique_new_labels}")
            # Update predictions
            final_labels = predicted_labels.copy()
            for i, idx in enumerate(idx_incorrectly_classified):
                final_labels[idx] = new_labels[i]
            predicted_labels = final_labels
            print(f"Assigned {len(np.unique(new_labels))} new classes to uncertain samples")
    print("===============================================================")
    if labels is not None and len(labels.shape) > 1:
        labels = labels.ravel()
    if len(predicted_labels.shape) > 1:
        predicted_labels = predicted_labels.ravel()

    return labels, predicted_labels

###

### Section for evaluating NELDA with your data. **This code is the first step**

In [13]:
'''
NELDA
If you wish to evaluate your data with NELDA, please modify this section of code.
'''

#data_flag = 'pathmnist'
data_flag = 'bloodmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

#flag = "pathmnist"
info_pathmnist = INFO[data_flag]
DataClass_pathmnist = getattr(medmnist, info_pathmnist['python_class'])
data_transform = transforms.Compose([
      transforms.ToTensor(),
])
train_data = DataClass_pathmnist(split='train', transform=data_transform, download=True)
val_data = DataClass_pathmnist(split='val', transform=data_transform, download=True)
test_data = DataClass_pathmnist(split='test', transform=data_transform, download=True)

BATCH_SIZE = 128
train_dl = data.DataLoader(train_data, batch_size= BATCH_SIZE, shuffle = True)
val_dl = data.DataLoader(val_data, batch_size= BATCH_SIZE, shuffle = False)
test_dl = data.DataLoader(test_data, batch_size= BATCH_SIZE, shuffle = False)

MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/


100%|██████████| 35.5M/35.5M [00:00<00:00, 95.7MB/s]


### This is the second step to generate classification values

In [23]:
# Test the exported NELDA models
import joblib
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np


# Load models
print("Loading models...")
models_dir = Path('models/ensemble')
cnn_path = models_dir / 'DeepGA.joblib'
lda_path = models_dir / 'lda.joblib'
#gmm_path = models_dir / 'gmm.joblib'

if not cnn_path.exists():
    raise FileNotFoundError(f"CNN model not found at {cnn_path}")
if not lda_path.exists():
    raise FileNotFoundError(f"LDA model not found at {lda_path}")

# Load and configure models
DeepGA_model = None
if cnn_path.exists():
    DeepGA_model = joblib.load(cnn_path)
    print("Loaded DeepGA model")
    DeepGA_model.eval()  # Ensure model is in evaluation mode
else:
    print("No DeepGA model found")

lda_model = None
if lda_path.exists():
    lda_model = joblib.load(lda_path)
    print("Loaded LDA model")
else:
    print("No LDA model found")

# Test with full test set
print("\nTesting with full test set...")
full_origin, full_preds = predict_ensemble(train_dl, DeepGA_model, lda_model, gmm_model)
print(f"Full dataset size: {len(full_preds)}")
print(f"Full test accuracy: {(full_preds == full_origin).mean():.4f}")

# Ensure 1D arrays
y_true = np.asarray(full_origin).ravel()
y_pred = np.asarray(full_preds).ravel()

accuracy = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 score (macro): {f1_macro:.4f}")
print(f"F1 score (weighted): {f1_weighted:.4f}")

# Optional: detailed per-class report
#print("\nClassification report:\n", classification_report(y_true, y_pred, digits=4))

Loading models...
Loaded DeepGA model
Loaded LDA model

Testing with full test set...
=======================   NELDA   =============================
Found 8 unique labels in confident predictions
Labels founds: [0 1 2 3 4 5 6 8]
Confident labels found by NELDA represent the 75.86754745380048 %
Untrusted tags found by NELDA represent the 24.132452546199517 %
Domain adaptation: Analyzing data to identify clusters and assign labels.
NELDA info: Some of your data is not within the distribution of the data with which NELDA was trained. 
 It was adjusted into 4 groups, and the following labels were assigned.
Labels assigned: [100 101 102 103]
Assigned 4 new classes to uncertain samples
Full dataset size: 11959
Full test accuracy: 0.0523
Accuracy: 0.0523
F1 score (macro): 0.0240
F1 score (weighted): 0.0403
